In [2]:
class Config:
    """
    All configuration parameters for the Launch Brand Dynamics simulation.
    """
    # Basic simulation parameters
    N_HCPS = 30000  # Number of healthcare providers
    N_WEEKS = 78    # Number of weeks to simulate
    
    # How HCPs are distributed across volume tiers at start
    VOLUME_TIER_PROPORTIONS = {
        0: 0.20,  # 25% in tier 0 (lowest volume)
        1: 0.25,  # 30% in tier 1
        2: 0.20,  # 25% in tier 2
        3: 0.20,  # 15% in tier 3
        4: 0.10,  # 5% in tier 4
        5: 0.05   # 0% in tier 5 (highest volume, critical tier)
    }
    
    # How HCP practices change week to week
    PRACTICE_CHANGE_PROBS = {
        'stable': 0.95,   # 95% stay the same
        'growth': 0.025,  # 2.5% grow (move up a tier)
        'decline': 0.025  # 2.5% decline (move down a tier)
    }
    
    # Average prescriptions per week by tier
    MARKET_NBRX_RATES_BY_TIER = {
        0: 0 / 12,    # Tier 0: 0 per year
        1: 1 / 12,    # Tier 1: 1 per year
        2: 3 / 12,    # Tier 2: 3 per year
        3: 8 / 12,    # Tier 3: 8 per year
        4: 30 / 12,   # Tier 4: 30 per year
        5: 65 / 12    # Tier 5: 65 per year
    }
    
    # How much this week's prescriptions depend on last week
    NBRX_CORRELATION = 0.3
    
    # Sales Reps parameters
    N_SALES_REPS = 150
    CALLS_PER_REP_PER_WEEK = 35
    TOTAL_WEEKLY_CALLS = N_SALES_REPS * CALLS_PER_REP_PER_WEEK
    
    # Sales reps use 12-week old data for targeting
    TARGETING_LAG_WEEKS = 12
    
    # Probability of successfully reaching HCP by tier
    ACCESS_PROBS_BY_TIER = {
        0: 1.0,   # Tier 0: Always accessible
        1: 1.0,   # Tier 1: Always accessible
        2: 0.8,   # Tier 2: 80% accessible
        3: 0.6,   # Tier 3: 60% accessible
        4: 0.3,   # Tier 4: 30% accessible
        5: 0.2    # Tier 5: 20% accessible
    }
    
    # HCP personality types and their distribution
    PERSONA_DISTRIBUTION = {
        'Advocate': 0.15,       # 15% are early adopters
        'Learner': 0.25,        # 25% want education first
        'Cost Sensitive': 0.20, # 20% focus on cost
        'Patient Driven': 0.25, # 25% wait for patient demand
        'Conservative': 0.15    # 15% are slow to change
    }
    
    # Market parameters
    N_EXISTING_BRANDS = 4  # Brands already in market
    NEW_BRAND_STRENGTH = 1.5  # Relative strength (1-2 scale)
    
    # How each persona adopts the new brand
    PERSONA_BEHAVIORS = {
        'Advocate': {
            'scaler': 2.0,          # High initial interest
            'adoption_week': 1,     # Adopts immediately
            'magnitude': 1.8,       # Moderate response to calls
            'recency': 18           # Remembers calls for 18 weeks
        },
        'Learner': {
            'scaler': 1.5,          # Good initial interest
            'adoption_week': 6,     # Needs some education
            'magnitude': 1.7,       # Good response to calls
            'recency': 12           # Remembers calls for 12 weeks
        },
        'Cost Sensitive': {
            'scaler': 0.3,          # Low initial interest
            'adoption_week': 26,    # Waits for coverage
            'magnitude': 4.0,       # High response when ready
            'recency': 8            # Short memory
        },
        'Patient Driven': {
            'scaler': 1.2,          # Moderate interest
            'adoption_week': 12,    # Waits for patient demand
            'magnitude': 2.0,       # Good response to calls
            'recency': 8            # Short memory
        },
        'Conservative': {
            'scaler': 0.5,          # Low initial interest
            'adoption_week': 48,    # Very slow to adopt
            'magnitude': 3.0,       # Needs lots of convincing
            'recency': 8            # Short memory
        }
    }
    
    # Insurance coverage rollout schedule
    PAYER_ACCESS_SCHEDULE = [
        (12, 0.10),      # Week 12: 10% coverage
        (26, 0.30),      # Week 26: 30% coverage
        (52, 0.60),      # Week 52: 60% coverage
        (float('inf'), 0.85)  # After week 52: 85% coverage
    ]

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [5]:
config = Config()
random_gen = np.random.default_rng(seed=42)  # Set a random seed for reproducibility

In [9]:
tier_choices = list(config.VOLUME_TIER_PROPORTIONS.keys())
tier_proportions = list(config.VOLUME_TIER_PROPORTIONS.values())

print("Tier Choices:", tier_choices)
print("Tier Proportions:", tier_proportions)

initial_tier = random_gen.choice(tier_choices, 
    size=config.N_HCPS,
    p=tier_proportions)

initial_tier_counts = pd.Series(initial_tier).value_counts(normalize = True).sort_index()
initial_tier_counts

Tier Choices: [0, 1, 2, 3, 4, 5]
Tier Proportions: [0.2, 0.25, 0.2, 0.2, 0.1, 0.05]


0    0.199200
1    0.250933
2    0.201733
3    0.196433
4    0.101500
5    0.050200
Name: proportion, dtype: float64

In [11]:
persona_choices = list(config.PERSONA_DISTRIBUTION.keys())
persona_proportions = list(config.PERSONA_DISTRIBUTION.values())

print("Persona Choices:", persona_choices)
print("Persona Proportions:", persona_proportions)

personas = random_gen.choice(persona_choices,
    size=config.N_HCPS,
    p=persona_proportions)

persona_counts = pd.Series(personas).value_counts(normalize=True).sort_index()
print("Persona Counts:\n", persona_counts)

Persona Choices: ['Advocate', 'Learner', 'Cost Sensitive', 'Patient Driven', 'Conservative']
Persona Proportions: [0.15, 0.25, 0.2, 0.25, 0.15]
Persona Counts:
 Advocate          0.150467
Conservative      0.152200
Cost Sensitive    0.197833
Learner           0.249433
Patient Driven    0.250067
Name: proportion, dtype: float64


In [14]:
hcps = pd.DataFrame({
    'hcp_id': range(config.N_HCPS),
    'initial_tier': initial_tier,
    'persona': personas
})

for persona, persona_params in config.PERSONA_BEHAVIORS.items():
    print(f"Persona: {persona}")
    print(f"  Scaler: {persona_params['scaler']}")
    print(f"  Adoption Week: {persona_params['adoption_week']}")
    print(f"  Magnitude: {persona_params['magnitude']}")
    print(f"  Recency: {persona_params['recency']}")
    print()

for persona, persona_params in config.PERSONA_BEHAVIORS.items():
    mask = hcps['persona'] == persona
    hcps.loc[mask, 'scaler'] = persona_params['scaler']
    hcps.loc[mask, 'adoption_week'] = persona_params['adoption_week']
    hcps.loc[mask, 'magnitude'] = persona_params['magnitude']
    hcps.loc[mask, 'recency'] = persona_params['recency']

hcps

Persona: Advocate
  Scaler: 2.0
  Adoption Week: 1
  Magnitude: 1.8
  Recency: 18

Persona: Learner
  Scaler: 1.5
  Adoption Week: 6
  Magnitude: 1.7
  Recency: 12

Persona: Cost Sensitive
  Scaler: 0.3
  Adoption Week: 26
  Magnitude: 4.0
  Recency: 8

Persona: Patient Driven
  Scaler: 1.2
  Adoption Week: 12
  Magnitude: 2.0
  Recency: 8

Persona: Conservative
  Scaler: 0.5
  Adoption Week: 48
  Magnitude: 3.0
  Recency: 8



,hcp_id,initial_tier,persona,scaler,adoption_week,magnitude,recency
0,0,2,Patient Driven,1.2,12.0,2.0,8.0
1,1,0,Patient Driven,1.2,12.0,2.0,8.0
2,2,3,Cost Sensitive,0.3,26.0,4.0,8.0
3,3,1,Patient Driven,1.2,12.0,2.0,8.0
4,4,5,Patient Driven,1.2,12.0,2.0,8.0
...,...,...,...,...,...,...,...
29995,29995,1,Patient Driven,1.2,12.0,2.0,8.0
29996,29996,4,Cost Sensitive,0.3,26.0,4.0,8.0
29997,29997,4,Learner,1.5,6.0,1.7,12.0
29998,29998,1,Patient Driven,1.2,12.0,2.0,8.0


In [32]:
weeks = np.arange(0, config.N_WEEKS)
hcp_ids = np.arange(0, config.N_HCPS)

weekly_df = pd.merge(
    pd.DataFrame({'week': weeks}),
    pd.DataFrame({'hcp_id': hcp_ids}),
    how='cross'
)

weekly_df['volume_tier'] = 0
weekly_df['market_nbrx'] = 0.0

weekly_df

,week,hcp_id,volume_tier,market_nbrx
0,0,0,0,0.0
1,0,1,0,0.0
2,0,2,0,0.0
3,0,3,0,0.0
4,0,4,0,0.0
...,...,...,...,...
2339995,77,29995,0,0.0
2339996,77,29996,0,0.0
2339997,77,29997,0,0.0
2339998,77,29998,0,0.0


In [33]:
week_0_mask = weekly_df['week'] == 0
weekly_df.loc[week_0_mask, 'volume_tier'] = hcps['initial_tier'].values

tier_to_nbrx_rate = weekly_df.loc[week_0_mask, 'volume_tier'].map(
    config.MARKET_NBRX_RATES_BY_TIER
)

# print("Tier to NBRx Rate Mapping:")
# print(tier_to_nbrx_rate)

weekly_df.loc[week_0_mask, 'market_nbrx'] = random_gen.poisson(
    tier_to_nbrx_rate
)

weekly_df

,week,hcp_id,volume_tier,market_nbrx
0,0,0,2,0.0
1,0,1,0,0.0
2,0,2,3,0.0
3,0,3,1,0.0
4,0,4,5,4.0
...,...,...,...,...
2339995,77,29995,0,0.0
2339996,77,29996,0,0.0
2339997,77,29997,0,0.0
2339998,77,29998,0,0.0


In [34]:
for week in range(1, config.N_WEEKS):
    prev_week = weekly_df[weekly_df['week'] == week - 1].copy()
    prev_week = prev_week.set_index('hcp_id')

    current_week_mask = weekly_df['week'] == week

    practice_changes = random_gen.choice(
        list(config.PRACTICE_CHANGE_PROBS.keys()),
        size=config.N_HCPS,
        p=list(config.PRACTICE_CHANGE_PROBS.values())
    )

    new_tiers = prev_week['volume_tier'].copy()

    growth_mask = practice_changes == 'growth'
    decline_mask = practice_changes == 'decline'

    new_tiers[growth_mask] = np.minimum(new_tiers[growth_mask] + 1, max(tier_choices))
    new_tiers[decline_mask] = np.maximum(new_tiers[decline_mask] - 1, min(tier_choices))

    weekly_df.loc[current_week_mask, 'volume_tier'] = new_tiers.values

    current_tier_to_nbrx_rate = weekly_df.loc[current_week_mask, 'volume_tier'].map(
        config.MARKET_NBRX_RATES_BY_TIER  )

    # print("Current Tier to NBRx Rate Mapping:")
    # print(current_tier_to_nbrx_rate)

    new_rx = random_gen.poisson(current_tier_to_nbrx_rate)
    prev_rx = prev_week['market_nbrx'].values
    corr_factor = config.NBRX_CORRELATION
    market_nbrx = (corr_factor * prev_rx + 
                (1 - corr_factor) * new_rx)

    weekly_df.loc[current_week_mask, 'market_nbrx'] = market_nbrx

    # Round prescriptions to integers
weekly_df['market_nbrx'] = weekly_df['market_nbrx'].round().astype(int)

weekly_df


,week,hcp_id,volume_tier,market_nbrx
0,0,0,2,0
1,0,1,0,0
2,0,2,3,0
3,0,3,1,0
4,0,4,5,4
...,...,...,...,...
2339995,77,29995,2,0
2339996,77,29996,4,3
2339997,77,29997,3,1
2339998,77,29998,5,7


In [39]:
# weekly_df.summary = weekly_df.groupby('week').agg({
#     'hcp_id': 'count',
#     'market_nbrx': 'sum',
#     'volume_tier': 'mean'
# }).reset_index()

# weekly_df.describe()
# weekly_df.info()

In [40]:
weekly_df['calls'] = 0

tier_by_week = weekly_df.pivot(
    index = 'hcp_id',
    columns = 'week',
    values = 'volume_tier'
)

tier_by_week

week,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
hcp_id,,,,,,,,,,,,,,,,,,,,,
0,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,3,3,3,2
1,0,0,0,0,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,3,3,3,3,3,3,3,3,3,3,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,5,5,5,5,5,5,5,5,5,5,...,3,3,3,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,1,1,1,1,1,1,1,1,1,2,...,2,2,2,2,2,2,2,2,2,2
29996,4,4,4,3,3,3,3,3,3,3,...,4,4,4,4,4,4,4,4,4,4
29997,4,4,4,4,4,4,4,4,4,4,...,3,3,3,3,3,3,3,3,3,3


In [ ]:
for week in range(config.N_WEEKS):
    calls_remaining = config.TOTAL_WEEKLY_CALLS

    data_week = 
